<a href="https://colab.research.google.com/github/timeseriesAI/tsai/blob/master/tutorial_nbs/01_Intro_to_Time_Series_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pathlib
current_dir = pathlib.Path.cwd()
parent_dir = current_dir.parent
preprocessing_output_dir = pathlib.Path('{}/1_preprocessing_all_domain/output'.format(parent_dir))
output_dir = pathlib.Path('{}/output'.format(current_dir))
pathlib.Path.mkdir(output_dir, mode=0o777, exist_ok=True)
print(current_dir)

In [ ]:
with open('{}/0_importsql/abnormal_list.txt'.format(parent_dir), 'r') as f:
    data = f.read()
outcome_name_list = data.splitlines()

In [ ]:
def split_x_y_data(df, OBP) :
    import numpy as np
    import pandas as pd

    y_data = df['label'].T.reset_index(drop=True) #df['label'].T.drop_duplicates().T.reset_index(drop=True)
    y_data = np.array(y_data)
    y_data = y_data[0:len(y_data):OBP].reshape(-1, 1).astype(int)
    #print(len(y_data), file=_logfile_)

    x_df = df.drop('label', axis=1)

    # 2-d data to 3-d data
    timestamp = OBP 
    x_data = np.array(x_df)
    x_data = x_data.reshape(-1, timestamp, x_data.shape[1]) # -1(sample), timestamp, column
    #x_data.shape, y_data.shape

    # get Column data
    new_col = x_df.columns
    print(x_data.shape, y_data.shape, len(new_col))
    return x_data, y_data, new_col

In [ ]:
import os
import pickle
from tqdm import tqdm
from Auto_lstm_attention import *
c = Auto_lstm_attention()

outcome_name = outcome_name_list[0]
# 결과물 저장할 폴더 생성
output_domain_path = pathlib.Path('{}/{}'.format(output_dir, outcome_name))
pathlib.Path.mkdir(output_domain_path, mode=0o777, parents=True, exist_ok=True)
        
concat_df = pd.read_csv('{}/{}.txt'.format(preprocessing_output_dir, outcome_name), index_col=False)

# ##### Case 1 : Split by subject_id #####
# id_data = concat_df[['subject_id', 'label']].drop_duplicates().reset_index(drop=True)
# x_id_data = np.array(id_data['subject_id'])
# y_id_data = np.array(id_data['label'])

# x_id_train, x_id_test, y_id_train, y_id_test = train_test_split(x_id_data, y_id_data, test_size=0.3, random_state=1, stratify=y_id_data) 

# train_df = concat_df[concat_df['subject_id'].isin(x_id_train)].reset_index(drop=True)
# test_df = concat_df[concat_df['subject_id'].isin(x_id_test)].reset_index(drop=True)

# train_df.to_csv('{}/{}_train.txt'.format(output_domain_path, outcome_name), index=False)
# test_df.to_csv('{}/{}_test.txt'.format(output_domain_path, outcome_name), index=False)

# concat_df = concat_df.drop(['subject_id', 'unique_id', 'cohort_start_date', 'concept_date', 'first_abnormal_date'], axis=1)
# train_df = train_df.drop(['subject_id', 'unique_id', 'cohort_start_date', 'concept_date', 'first_abnormal_date'], axis=1)
# test_df = test_df.drop(['subject_id', 'unique_id', 'cohort_start_date', 'concept_date', 'first_abnormal_date'], axis=1)

# x_data, y_data, new_col = split_x_y_data(concat_df, OBP=28)
# x_train, y_train, new_col = split_x_y_data(train_df, OBP=28)
# x_test, y_test, new_col = split_x_y_data(test_df, OBP=28)

#### Case 2 : Split ignore subject_id #####
concat_df = concat_df.drop(['subject_id', 'unique_id', 'cohort_start_date', 'concept_date', 'first_abnormal_date'], axis=1)
x_data, y_data, new_col = split_x_y_data(concat_df, OBP=28)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=1, stratify=y_data) 


In [ ]:
# ## NOTE: UNCOMMENT AND RUN THIS CELL IF YOU NEED TO INSTALL/ UPGRADE TSAI
# stable = False # True: stable version in pip, False: latest version from github
# if stable: 
#     !pip install tsai -U >> /dev/null
# else:      
#     !pip install git+https://github.com/timeseriesAI/tsai.git -U >> /dev/null
# ## NOTE: REMEMBER TO RESTART (NOT RECONNECT/ RESET) THE KERNEL/ RUNTIME ONCE THE INSTALLATION IS FINISHED

In [ ]:
from tsai.all import*

In [ ]:
computer_setup()

In [ ]:
X_train, y_train, X_test, y_test  = get_UCR_data(dsid, return_split=True)

you can use this convenience function to get X, y and splits:

In [ ]:
splits

In [ ]:
x, y, splits = combine_split_data([x_train, x_test], [y_train, y_test])

In [ ]:
y.shape

### Prepare datasets ??

The first step is to create datasets. This is very easy to do in v2. 

In TS classification problems, you will usually want to use an item tfm to transform y into categories.

We'll use inplace=True to preprocess data at dataset initialization. This will significantly speed up training. 

In [ ]:
tfms  = [None, [Categorize()]]
dsets = TSDatasets(x, y.flatten(), tfms=tfms, splits=splits, inplace=True)

We'll now build the dataloaders that will create batches of data.

You will need to pass:

* datasets: usually 2 - train and valid -  or 1 - test or unlabeled- depending on the problem
* batch size(s): you may pass a single value (will will be appied to all dls, or different values, one for each dl.
* batch_tfms (same as after_batch): you may decide to pass some tfms at the batch level. In this case for example, we'll standardize the data (0 mean and 1 std). You may get more details on how these transforms work in the transforms nb.
* num workers: num_workers > 0 is used to preprocess batches of data so that the next batch is ready for use when the current batch has been finished. More num_workers would consume more memory usage but is helpful to speed up the I/O process. This will depend on your machine, dataset, etc. You may want to start with 0, and test other values to see how to train faster. For me, 0 works better.

In [ ]:
dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[32, 64], batch_tfms=[TSStandardize()], num_workers=0)

### Visualize data

In [ ]:
dls.show_batch(sharey=True)

## Build learner ??

In [ ]:
model = InceptionTime(dls.vars, dls.c)
learn = Learner(dls, model, metrics=accuracy)
learn.save('stage0')

## Train model ?????

### LR find ??

In [ ]:
learn.load('stage0')
learn.lr_find()
#learn.lr_find(suggestions=False)

### Train ?????��?

In [ ]:
learn.fit_one_cycle(25, lr_max=1e-3)
learn.save('stage1')

In [ ]:
learn.recorder.plot_metrics()

Let's pretend we need to end the working session now for some reason, but we'd like to continue working with this datasets and learner in the future. 

To save everything you can use a convenience function I've created that saves the learner with the model, the data and the opt function status: 

In [ ]:
learn.save_all(path='export', dls_fname='dls', model_fname='model', learner_fname='learner')

As soon as we've done this, we can end the session, and continue at any time in the future. 

Let's simulate that we need to end the session now:

In [ ]:
del learn, dsets, dls

Next time we go back to work, we'll need to reload the datasets and learner (with the same status we had):

In [ ]:
learn = load_learner_all(path='export', dls_fname='dls', model_fname='model', learner_fname='learner')
dls = learn.dls
valid_dl = dls.valid
b = next(iter(valid_dl))
b

In [ ]:
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
valid_probas, valid_targets, valid_preds

We can confirm the learner has the same status it had at the end of training, by confirming the validation accuracy is the same:

In [ ]:
(valid_targets == valid_preds).float().mean()

Great! It's the same. This means we have now the learner at the same point where we left it.

## Visualize results ??

In [ ]:
learn.show_results()

In [ ]:
learn.show_probas()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.most_confused(min_val=3)

## Inference on additional data ??

Let's say we want to predict labels on new data. Let's see how this works.

We may have additional data (test set) where we want to check our performance. In this case, we'd add a labeled dataset:

In [ ]:
# Labeled test data
test_ds = valid_dl.dataset.add_test(x, y.flatten())# In this case I'll use X and y, but this would be your test data
test_dl = valid_dl.new(test_ds)
next(iter(test_dl))

By selecting the valid dataset (valid_dl.dataset) we ensure that the same tfms applied to the valid data will be applied to the new data.

In [ ]:
test_probas, test_targets, test_preds = learn.get_preds(dl=test_dl, with_decoded=True, save_preds=None, save_targs=None)
test_probas, test_targets, test_preds

In [ ]:
print(f'accuracy: {skm.accuracy_score(test_targets, test_preds):10.6f}')

If data is unlabeled, we'd just do this: 

In [ ]:
# Unlabeled data
test_ds = dls.dataset.add_test(x)
test_dl = valid_dl.new(test_ds)
next(iter(test_dl))

In [ ]:
test_probas, *_ = learn.get_preds(dl=test_dl, save_preds=None)
test_probas

## Summary ?

This is all the code you need to train a TS model. As you can see, it's v2 is easier to use and faster compared to v1.

In [ ]:
dsid = 'NATOPS' 
X, y, splits = get_UCR_data(dsid, return_split=False)
tfms  = [None, [Categorize()]]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits, inplace=True)
dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[64, 128], batch_tfms=[TSStandardize()], num_workers=0)
model = InceptionTime(dls.vars, dls.c)
learn = Learner(dls, model, metrics=accuracy)
learn.fit_one_cycle(25, lr_max=1e-3)
learn.plot_metrics()

## New scikit-learn-like API ??

As of `tsai` version 0.2.15 I have added a new scikit-learn-like API to further simplify the learner creation. 

I will prepare a new tutorial to further demonstrate how you can use the new API.

This is how you can use it for Time Series Classification: 

In [ ]:
dsid = 'NATOPS' 
X, y, splits = get_UCR_data(dsid, return_split=False)
learn = TSClassifier(X, y, splits=splits, bs=[64, 128], batch_tfms=[TSStandardize()], arch=InceptionTime, metrics=accuracy)
learn.fit_one_cycle(25, lr_max=1e-3)
learn.plot_metrics()